In [1]:

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

import org.apache.spark.mllib.linalg.{Vector=>OldVector}


In [2]:
val spark = SparkSession
      .builder
      .master("local[*]")
      .getOrCreate

spark = org.apache.spark.sql.SparkSession@5ee53095


org.apache.spark.sql.SparkSession@5ee53095

In [3]:
spark

org.apache.spark.sql.SparkSession@5ee53095

In [4]:
spark.stop()

In [ ]:
var train = spark
    .read
    .format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("inferSchema", "true")
    .load("/labs/slaba05/lab05_train.csv")
    .drop("_c0")
    .cache

In [ ]:
var test = spark
    .read
    .format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("inferSchema", "true")
    .load("/labs/slaba05/lab05_test.csv")
    .drop("_c0")
    .na.fill(0)
    .cache

In [ ]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"

In [ ]:
train = train.drop(categoriesCols:_*).na.fill(0)

In [ ]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(train.drop("TARGET").columns)
    .setOutputCol("features")

In [ ]:
val trainCleaned = vectorAssembler.transform(train).cache

In [ ]:
val gbt = new GBTClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setMaxIter(10)
    .setFeatureSubsetStrategy("auto")
    .fit(trainCleaned)

In [ ]:
val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

In [ ]:
val predictions = gbt.transform(vectorAssembler.transform(test)).cache

In [ ]:
predictions.show()

In [ ]:
predictions
    .withColumn("target", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "target")
    .coalesce(1)
    .write
    .option("header","true")
    .option("sep" ,"\t")
    .mode("overwrite")
    .format("csv")
    .save("/user/sergey.vaschuk/lab05.csv")

Name: Compile Error
Message: <console>:39: error: not found: value pwd
       ! pwd
         ^

StackTrace: 